# Step 9: V06 — Nearby eclipsing binary search (catalog)

Goal: check whether there are known eclipsing binaries near the target position that could be contaminating the TESS aperture.

This is a catalog-context check (network required).


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(
    stitched=lc,
    candidate=candidate,
    network=True,
    ra_deg=sh.RA_DEG,
    dec_deg=sh.DEC_DEG,
)

r = session.run('V06')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': r.flags,
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "dec": 43.36068888888889,
    "n_ebs_found": 0,
    "ra": 308.5672083333333,
    "search_radius_arcsec": 42.0,
    "status": "ok"
  },
  "status": "ok"
}
```

</details>


In [ ]:
# Plot V06: Nearby eclipsing binaries
out = {
    'status': r.status,
    'flags': r.flags,
}

try:
    import matplotlib.pyplot as plt
    from bittr_tess_vetter.api import plot_nearby_ebs
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE and (r.status == 'ok') and getattr(r, 'raw', None) and (r.raw or {}).get('plot_data'):
    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='09_v06_nearby_ebs')
    run_path = run_out_dir / 'V06_nearby_ebs.png'
    docs_path = (docs_out_dir / 'V06_nearby_ebs.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(6, 6))
    plot_nearby_ebs(r, ax=ax)
    ax.set_title('V06: Nearby eclipsing binaries (catalog)')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/09_v06_nearby_ebs/V06_nearby_ebs.png`

![V06: Nearby eclipsing binaries](../artifacts/tutorial_toi-5807-incremental/09_v06_nearby_ebs/V06_nearby_ebs.png)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/09_v06_nearby_ebs/V06_nearby_ebs.png",
  "flags": [],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/09_v06_nearby_ebs/V06_nearby_ebs.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** n_ebs_found = 0 within 42" of the target coordinates.
- **Why it’s useful:** removes an easy false-positive pathway (known catalog EB contaminating the aperture).
- **Interpretation:** no obvious catalog EB contaminant nearby; still need pixel-level localization.
- **Next step:** V07 (ExoFOP TOI lookup for community context).

</details>
